In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import nibabel as nib
import pandas as pd

import os
import shutil
from torch.utils.data import DataLoader
import torchio as tio


In [4]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda:0')
else:
    DEVICE = torch.device('cpu')
    
print("Using PyTorch version:", torch.__version__, 'Device: ', DEVICE)
    

Using PyTorch version: 1.12.0+cu116 Device:  cuda:0


In [5]:
path2xls = './IXI.xls'
labels_df = pd.read_excel(path2xls)
labels_df.head()

,IXI_ID,"SEX_ID (1=m, 2=f)",HEIGHT,WEIGHT,ETHNIC_ID,MARITAL_ID,OCCUPATION_ID,QUALIFICATION_ID,DOB,DATE_AVAILABLE,STUDY_DATE,AGE
0,1,1,170,80,2,3,5,2,1968-02-22,0,NaT,NaN
1,2,2,164,58,1,4,1,5,1970-01-30,1,2005-11-18,35.800137
2,12,1,175,70,1,2,1,5,1966-08-20,1,2005-06-01,38.781656
3,13,1,182,70,1,2,1,5,1958-09-15,1,2005-06-01,46.710472
4,14,2,163,65,1,4,1,5,1971-03-15,1,2005-06-09,34.236824


In [6]:
sex_df = labels_df[['IXI_ID', 'SEX_ID (1=m, 2=f)']]
sex_df

,IXI_ID,"SEX_ID (1=m, 2=f)"
0,1,1
1,2,2
2,12,1
3,13,1
4,14,2
...,...,...
614,652,1
615,653,1
616,655,1
617,660,1


In [7]:
path_dir = './IXI'
file_list = os.listdir(path_dir)
file_list

['swc1nIXI016-Guys-0697-IXI3DMPRAG_-s231_-0301-00003-000001-01_RAS_denoised.nii',
 'swc1nIXI138-Guys-0746-IXI3DMPRAG_-s240_-0401-00004-000001-01_RAS_denoised.nii',
 'swc1nIXI077-Guys-0752-IXI3DMPRAG_-s242_-0301-00003-000001-01_RAS_denoised.nii',
 'swc1nIXI468-Guys-0985-MPRAGESEN_-s417_-0301-00003-000001-01_RAS_denoised.nii',
 'swc1nIXI629-Guys-1095-MPRAGESEN_-sIXI62_-0301-00003-000001-01_RAS_denoised.nii',
 'swc1nIXI496-Guys-1045-MPRAGESEN_-s421_-0301-00003-000001-01_RAS_denoised.nii',
 'swc1nIXI639-Guys-1088-MPRAGESEN_-s445_-0301-00003-000001-01_RAS_denoised.nii',
 'swc1nIXI336-Guys-0904-MPRAGESEN_-s306_-0301-00003-000001-01_RAS_denoised.nii',
 'swc1nIXI420-Guys-1028-MPRAGESEN_-s424_-0301-00003-000001-01_RAS_denoised.nii',
 'swc1nIXI307-IOP-0872-SAGFSPGR_-sIXI30_-0003-00001-000001-01_RAS_denoised.nii',
 'swc1nIXI219-Guys-0894-MPRAGESEN_-s304_-0301-00003-000001-01_RAS_denoised.nii',
 'swc1nIXI303-IOP-0968-SAGFSPGR_-sIXI30_-0003-00001-000001-01_RAS_denoised.nii',
 'swc1nIXI178-Guys-0778

In [8]:

def indexing(x):
    return int(x[8:11])
    
    

In [9]:
f_list=[]
for f in file_list:
    x = indexing(f)
    f_list.append(x)
    
f_list
    

[16,
 138,
 77,
 468,
 629,
 496,
 639,
 336,
 420,
 307,
 219,
 303,
 178,
 423,
 309,
 279,
 344,
 153,
 123,
 536,
 331,
 143,
 474,
 450,
 528,
 268,
 170,
 233,
 87,
 587,
 640,
 368,
 222,
 431,
 193,
 44,
 76,
 158,
 119,
 27,
 495,
 114,
 662,
 109,
 200,
 188,
 300,
 393,
 442,
 576,
 351,
 461,
 573,
 597,
 164,
 574,
 207,
 26,
 192,
 28,
 413,
 594,
 63,
 621,
 294,
 399,
 498,
 641,
 84,
 232,
 196,
 417,
 332,
 24,
 493,
 411,
 50,
 552,
 37,
 209,
 55,
 390,
 315,
 401,
 548,
 17,
 113,
 154,
 595,
 45,
 652,
 342,
 653,
 36,
 265,
 500,
 64,
 182,
 115,
 415,
 549,
 388,
 463,
 23,
 112,
 488,
 308,
 359,
 433,
 20,
 408,
 365,
 418,
 381,
 364,
 469,
 430,
 370,
 288,
 241,
 454,
 35,
 330,
 371,
 43,
 480,
 416,
 644,
 157,
 625,
 118,
 73,
 464,
 426,
 183,
 648,
 456,
 409,
 230,
 348,
 592,
 166,
 324,
 61,
 558,
 172,
 41,
 491,
 523,
 551,
 312,
 305,
 503,
 197,
 434,
 224,
 517,
 74,
 375,
 400,
 563,
 120,
 406,
 326,
 285,
 186,
 100,
 286,
 25,
 199,
 266,
 

In [10]:
IXI_df1 = pd.DataFrame(zip(f_list, file_list), columns = ['IXI_ID', 'file_name'])
IXI_df2 = pd.merge(IXI_df1, sex_df, how='outer', on='IXI_ID')
IXI_df3 = IXI_df2.drop_duplicates(['IXI_ID']
                                  )
IXI_df4 = IXI_df3.dropna()
IXI_df4 = IXI_df4.drop('IXI_ID',axis=1)
IXI_df4 

,file_name,"SEX_ID (1=m, 2=f)"
0,swc1nIXI016-Guys-0697-IXI3DMPRAG_-s231_-0301-0...,1
1,swc1nIXI138-Guys-0746-IXI3DMPRAG_-s240_-0401-0...,2
2,swc1nIXI077-Guys-0752-IXI3DMPRAG_-s242_-0301-0...,2
3,swc1nIXI468-Guys-0985-MPRAGESEN_-s417_-0301-00...,1
4,swc1nIXI629-Guys-1095-MPRAGESEN_-sIXI62_-0301-...,1
...,...,...
399,swc1nIXI373-IOP-0967-SAGFSPGR_-sIXI37_-0003-00...,2
400,swc1nIXI428-Guys-0996-MPRAGESEN_-s419_-0301-00...,2
401,swc1nIXI531-Guys-1057-MPRAGESEN_-s427_-0301-00...,1
403,swc1nIXI458-Guys-0993-MPRAGESEN_-s418_-0301-00...,1


In [11]:

from torch.utils.data import Dataset

torch.manual_seed(0)

class PBA_Dataset(Dataset):
    def __init__(self, data_dir, dataframe ,  transform=None):
        path2data = data_dir
        
        filenames = dataframe['file_name'].to_list()
        
        self.full_filenames = [os.path.join(path2data, f) for f in filenames]
        
        labels_df = dataframe.copy()
        labels_df.set_index('file_name', inplace=True)
        
        self.labels = [labels_df.loc[filename].values[0] for filename in filenames]
        
        self.transform = transform
        
    def __len__(self):
        return len(self.full_filenames)
    def __getitem__(self, idx):
        image_3d = nib.load(self.full_filenames[idx])
        image = image_3d.get_fdata()
        
        image = np.expand_dims(image, axis=0)
        image = self.transform(image)
        label = self.labels[idx]
        return image, label
    
import torchvision.transforms as transforms
data_transformer = tio.Compose([tio.ZNormalization()])

data_dir = './IXI/'
    

In [12]:
training_transform = tio.Compose([
    tio.RandomBlur(p=0.25),
    tio.RandomNoise(p=0.25),
    tio.RandomMotion(p=0.2),
    tio.OneOf({
        tio.RandomAffine(),
        tio.RandomFlip(),
        tio.RandomBiasField()
    }),
    tio.ZNormalization()
])

In [13]:
from sklearn.model_selection import train_test_split

def get_data(data_file):
    data = data_file
    print("data.shape: {}".format(data.shape))

    trainingSet, test_df = train_test_split(data, test_size=0.2, random_state=45346)
    train_df, val_df = train_test_split(trainingSet, test_size=0.2, random_state=257572)

    print("Train.shape: {}, Val.shape: {}, Test.shape: {}".format(train_df.shape, val_df.shape, test_df.shape))

    data.loc[train_df.index, 'split_type'] = "Train"
    data.loc[val_df.index, 'split_type'] = "Val"
    data.loc[test_df.index, 'split_type'] = "Test"

    train_df = data.loc[train_df.index, :]
    val_df = data.loc[val_df.index, :]
    test_df = data.loc[test_df.index, :]

    return train_df, val_df, test_df


In [14]:
train_df, val_df, test_df = get_data(IXI_df4)

data.shape: (381, 2)
Train.shape: (243, 2), Val.shape: (61, 2), Test.shape: (77, 2)


In [15]:

val_data = PBA_Dataset(data_dir, val_df, data_transformer)
test_data = PBA_Dataset(data_dir, test_df, data_transformer)

In [16]:
train_data0 = PBA_Dataset(data_dir, train_df, training_transform)
train_data1 = PBA_Dataset(data_dir, train_df, training_transform)
train_data2 = PBA_Dataset(data_dir, train_df, training_transform)
train_data3 = PBA_Dataset(data_dir, train_df, training_transform)
train_data4 = PBA_Dataset(data_dir, train_df, training_transform)
train_data5 = PBA_Dataset(data_dir, train_df, training_transform)
train_data6 = PBA_Dataset(data_dir, train_df, training_transform)
train_data7 = PBA_Dataset(data_dir, train_df, training_transform)
train_data8 = PBA_Dataset(data_dir, train_df, training_transform)




In [17]:
train_data = torch.utils.data.ConcatDataset([train_data0,train_data1,train_data2,train_data3,train_data4,train_data5,train_data6,train_data7,train_data8])

In [18]:
len(train_data)

2187

In [19]:
from torch.utils.data import DataLoader

In [76]:
# Hyper parameters
NUM_EPOCHS = 500
BATCH_SIZE = 8
LEARNING_RATE = 0.001
IMAGE_SHAPE = (121, 145, 121)


In [77]:
trn_loader = DataLoader(train_data, shuffle=True, num_workers=4, batch_size=BATCH_SIZE, drop_last=True)
val_loader = DataLoader(val_data, shuffle=True, num_workers=4, batch_size=BATCH_SIZE, drop_last=True)
tst_loader = DataLoader(test_data, shuffle=True,batch_size=BATCH_SIZE)

In [78]:
print(trn_loader)

In [79]:
for (X_train, y_train) in trn_loader:

    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    print(y_train)
    break

X_train: torch.Size([8, 1, 121, 145, 121]) type: torch.FloatTensor
y_train: torch.Size([8]) type: torch.LongTensor
tensor([2, 2, 2, 1, 2, 2, 1, 2])


In [80]:
for (X_train, y_train) in val_loader:

    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    print(y_train)
    break

X_train: torch.Size([8, 1, 121, 145, 121]) type: torch.FloatTensor
y_train: torch.Size([8]) type: torch.LongTensor
tensor([2, 2, 1, 1, 2, 2, 1, 2])


In [114]:
class Model3D(nn.Module):
    def __init__(self):
        super(Model3D, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv3d(1, 8, 3, stride=1),
            nn.ReLU(),
            nn.Conv3d(8, 8, 3, stride=1),
            nn.BatchNorm3d(8),
            nn.ReLU(),
            nn.MaxPool3d(2, stride=2),
            nn.Dropout3d(0.7))

        self.block2 = nn.Sequential(
            nn.Conv3d(8, 16, 3, stride=1),
            nn.ReLU(),
            nn.Conv3d(16, 16, 3, stride=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d(2, stride=2),
            nn.Dropout3d(0.7))

        self.block3 = nn.Sequential(
            nn.Conv3d(16, 32, 3, stride=1),
            nn.ReLU(),
            nn.Conv3d(32, 32, 3, stride=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.MaxPool3d(2, stride=2),
            nn.Dropout3d(0.7))

        self.block4 = nn.Sequential(
            nn.Conv3d(32, 64, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv3d(64, 64, 3, stride=1, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.MaxPool3d(2, stride=2),
            nn.Dropout3d(0.7))

        self.block5 = nn.Sequential(
            nn.Conv3d(64, 128, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv3d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(),
            nn.MaxPool3d(2, stride=2),
            nn.Dropout3d(0.7))

        self.classifier2 = nn.Linear(1536, 3)


    def forward(self, x):
        x = self.block1(x)
        # print(x.shape)
        x = self.block2(x)
        # print(x.shape)
        x = self.block3(x)
        # print(x.shape)
        x = self.block4(x)
        # print(x.shape)
        x = self.block5(x)
        # print(x.shape)
        x = self.classifier2(x.view(-1, x.shape[1]*x.shape[2]*x.shape[3]*x.shape[4]))
        # print(x.shape)
        

        return F.log_softmax(x, dim=1)

In [115]:
model = Model3D().to(DEVICE)

In [116]:
model.load_state_dict(torch.load('./Adam_MSE_dropout_ag_znormal_dropout7_.pth'), strict=False)

_IncompatibleKeys(missing_keys=['classifier2.weight', 'classifier2.bias'], unexpected_keys=['classifier.weight', 'classifier.bias'])

In [117]:
for child in model.children():
    print(child)

Sequential(
  (0): Conv3d(1, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (1): ReLU()
  (2): Conv3d(8, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (3): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): ReLU()
  (5): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Dropout3d(p=0.7, inplace=False)
)
Sequential(
  (0): Conv3d(8, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (1): ReLU()
  (2): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (3): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): ReLU()
  (5): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Dropout3d(p=0.7, inplace=False)
)
Sequential(
  (0): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (1): ReLU()
  (2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (3): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): ReLU()
 

In [118]:
ct = 0
for child in model.children():
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

In [119]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)
criterion = torch.nn.CrossEntropyLoss()

In [120]:
print(model)

Model3D(
  (block1): Sequential(
    (0): Conv3d(1, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): ReLU()
    (2): Conv3d(8, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (3): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU()
    (5): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Dropout3d(p=0.7, inplace=False)
  )
  (block2): Sequential(
    (0): Conv3d(8, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): ReLU()
    (2): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (3): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): ReLU()
    (5): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Dropout3d(p=0.7, inplace=False)
  )
  (block3): Sequential(
    (0): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): ReLU()
    (2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (3): BatchNorm3

In [121]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

In [122]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = data.float().to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
  
        loss = criterion(output.squeeze(), target)
        
        loss.backward()
        optimizer.step()
        



In [126]:
def eval(model, test_loader):
    model.eval()
    test_loss=0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.float().to(DEVICE), target.to(DEVICE)
            output = model(data)
           
            
            test_loss += criterion(output.squeeze(), target).item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy
            
            

In [129]:
import time
import copy

def train_baseline(model, train_loader, val_loader,
                   optimizer, num_epochs = NUM_EPOCHS):
     
   
    
      for epoch in range(1, num_epochs + 1):
            since = time.time()
            train(model, train_loader, optimizer)
            train_loss, train_acc = eval(model, train_loader)
            writer.add_scalar("loss/train_eval", train_loss, epoch)
            val_loss, val_acc = eval(model, val_loader)
            writer.add_scalar("loss/val_eval", val_loss, epoch)

            if epoch % 10 == 0:
              for name, param in model.named_parameters():
                writer.add_histogram(name, param.clone().cpu().data.numpy(), epoch)
            time_elapsed = time.time() - since
            print('----------epoch {}----------'.format(epoch))
        
            print('train Loss: {:.4f}, Accuracy: {:.2f}%'
              .format(train_loss, train_acc))
            print('val Loss: {:.4f}, Accuracy: {:.2f}%'
              .format(val_loss, val_acc))
            print('Completed in {:.0f}m {:.0f}s'
              .format(time_elapsed // 60, time_elapsed % 60))
           
      return model



In [130]:
torch.cuda.empty_cache()
base = train_baseline(model, trn_loader, val_loader, optimizer, NUM_EPOCHS)
writer.flush()
writer.close()

----------epoch 1----------
train Loss: 0.0789, Accuracy: 64.11%
val Loss: 0.0781, Accuracy: 49.18%
Completed in 6m 2s
----------epoch 2----------
train Loss: 0.0786, Accuracy: 61.45%
val Loss: 0.0770, Accuracy: 50.82%
Completed in 6m 10s
----------epoch 3----------
train Loss: 0.0774, Accuracy: 66.26%
val Loss: 0.0770, Accuracy: 52.46%
Completed in 5m 52s
----------epoch 4----------
train Loss: 0.0810, Accuracy: 60.77%
val Loss: 0.0878, Accuracy: 50.82%
Completed in 6m 5s
----------epoch 5----------
train Loss: 0.0784, Accuracy: 63.65%
val Loss: 0.0810, Accuracy: 47.54%
Completed in 5m 53s
----------epoch 6----------
train Loss: 0.0888, Accuracy: 59.58%
val Loss: 0.0954, Accuracy: 50.82%
Completed in 5m 59s
----------epoch 7----------
train Loss: 0.0793, Accuracy: 63.74%
val Loss: 0.0867, Accuracy: 49.18%
Completed in 5m 41s
----------epoch 8----------
train Loss: 0.0801, Accuracy: 62.00%
val Loss: 0.0915, Accuracy: 47.54%
Completed in 5m 56s
----------epoch 9----------
train Loss: 0.

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd0856f2e60>
Traceback (most recent call last):
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1464, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd0856f2e60>
Traceback (most recent call last):
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py"

----------epoch 160----------
train Loss: 0.0796, Accuracy: 63.83%
val Loss: 0.0679, Accuracy: 65.57%
Completed in 6m 13s


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd0856f2e60>
Traceback (most recent call last):
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1464, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd0856f2e60>
Traceback (most recent call last):
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py"

----------epoch 161----------
train Loss: 0.0716, Accuracy: 70.78%
val Loss: 0.0689, Accuracy: 60.66%
Completed in 5m 55s


...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd0856f2e60>
Traceback (most recent call last):
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1464, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd0856f2e60>
Traceback (most recent call last):
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py"

----------epoch 183----------
train Loss: 0.0726, Accuracy: 70.28%
val Loss: 0.0751, Accuracy: 54.10%
Completed in 6m 11s


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd0856f2e60>
Traceback (most recent call last):
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1464, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fd0856f2e60>
Traceback (most recent call last):
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1481, in __del__
    self._shutdown_workers()
  File "/home/guest/Desktop/IXI/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py"

----------epoch 184----------
train Loss: 0.0716, Accuracy: 70.87%
val Loss: 0.0696, Accuracy: 57.38%
Completed in 5m 58s
----------epoch 185----------
train Loss: 0.1005, Accuracy: 59.30%
val Loss: 0.1198, Accuracy: 47.54%
Completed in 5m 59s
----------epoch 186----------
train Loss: 0.0719, Accuracy: 70.23%
val Loss: 0.0796, Accuracy: 55.74%
Completed in 5m 51s
----------epoch 187----------
train Loss: 0.0731, Accuracy: 69.87%
val Loss: 0.0745, Accuracy: 57.38%
Completed in 5m 58s
----------epoch 188----------
train Loss: 0.0756, Accuracy: 68.08%
val Loss: 0.0677, Accuracy: 65.57%
Completed in 5m 52s
----------epoch 189----------
train Loss: 0.0745, Accuracy: 70.74%
val Loss: 0.0689, Accuracy: 59.02%
Completed in 5m 49s
----------epoch 190----------
train Loss: 0.0729, Accuracy: 70.23%
val Loss: 0.0689, Accuracy: 57.38%
Completed in 5m 58s
----------epoch 191----------
train Loss: 0.0736, Accuracy: 69.27%
val Loss: 0.0747, Accuracy: 60.66%
Completed in 5m 49s
----------epoch 192-----

KeyboardInterrupt: 